# 05 - Customer Segmentation

**Customer Lifetime Value Prediction**

**Team:** The Starks
- Othmane Zizi (261255341)
- Fares Joni (261254593)
- Tanmay Giri (261272443)

This notebook segments customers based on predicted CLV and develops actionable recommendations.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sys
from pathlib import Path
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler

# Add src to path
sys.path.append(str(Path('../src').resolve()))
from data_loader import load_customer_features

plt.style.use('seaborn-v0_8-whitegrid')
pd.set_option('display.max_columns', None)

## 1. Load Data

In [ ]:
# Load customer features
features_df = load_customer_features('customer_features.csv')

# Load predictions (if modeling notebook was run)
predictions_path = Path('../data/processed/clv_predictions.csv')
if predictions_path.exists():
    predictions_df = pd.read_csv(predictions_path)
    df = features_df.merge(predictions_df[['Customer ID', 'Predicted_CLV']], on='Customer ID', how='left')
else:
    df = features_df.copy()
    df['Predicted_CLV'] = df['CLV']  # Use actual CLV if no predictions

print(f"Loaded {len(df)} customers")
df.head()

## 2. Quantile-Based CLV Segmentation

In [ ]:
# Segment customers into 4 tiers based on predicted CLV
clv_col = 'Predicted_CLV' if 'Predicted_CLV' in df.columns else 'CLV'

df['CLV_Segment'] = pd.qcut(
    df[clv_col].rank(method='first'), 
    q=4, 
    labels=['Low Value', 'Medium-Low', 'Medium-High', 'High Value']
)

# Segment distribution
segment_counts = df['CLV_Segment'].value_counts().sort_index()
print("Segment Distribution:")
for seg, count in segment_counts.items():
    print(f"  {seg}: {count} customers ({count/len(df)*100:.1f}%)")

In [ ]:
# Visualize segment distribution
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Pie chart
colors = ['#ff9999', '#ffcc99', '#99ccff', '#99ff99']
segment_counts.plot(kind='pie', ax=axes[0], colors=colors, autopct='%1.1f%%')
axes[0].set_title('Customer Distribution by CLV Segment')
axes[0].set_ylabel('')

# Bar chart with CLV values
segment_clv = df.groupby('CLV_Segment')[clv_col].sum().sort_index()
segment_clv.plot(kind='bar', ax=axes[1], color=colors, edgecolor='black')
axes[1].set_title('Total CLV by Segment')
axes[1].set_xlabel('Segment')
axes[1].set_ylabel('Total CLV (GBP)')
axes[1].tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.savefig('../reports/figures/segment_distribution.png', dpi=150, bbox_inches='tight')
plt.show()

# CLV contribution
total_clv = df[clv_col].sum()
print("\nCLV Contribution by Segment:")
for seg in segment_clv.index:
    pct = segment_clv[seg] / total_clv * 100
    print(f"  {seg}: £{segment_clv[seg]:,.0f} ({pct:.1f}%)")

## 3. Segment Profiling

In [ ]:
# Profile each segment
profile_cols = ['Recency', 'Frequency', 'Monetary', 'Tenure', 
                'AvgTimeBetweenPurchases', 'NumUniqueProducts', 
                'AvgBasketSize', 'AvgOrderValue', 'CLV', clv_col]

# Remove duplicates if Predicted_CLV is same as CLV
profile_cols = list(dict.fromkeys(profile_cols))

segment_profile = df.groupby('CLV_Segment')[profile_cols].mean()
print("Segment Profiles (Mean Values):")
segment_profile.T

In [ ]:
# Visualize segment profiles (radar chart style using bar charts)
metrics = ['Recency', 'Frequency', 'Monetary', 'NumUniqueProducts', 'AvgOrderValue']

fig, axes = plt.subplots(1, len(metrics), figsize=(20, 4))

for ax, metric in zip(axes, metrics):
    segment_profile[metric].plot(kind='bar', ax=ax, color=colors, edgecolor='black')
    ax.set_title(metric)
    ax.set_xlabel('')
    ax.tick_params(axis='x', rotation=45)

plt.suptitle('Segment Characteristics', y=1.02, fontsize=14)
plt.tight_layout()
plt.savefig('../reports/figures/segment_profiles.png', dpi=150, bbox_inches='tight')
plt.show()

## 4. RFM-Based Segmentation (Alternative)

In [ ]:
# Create RFM scores (1-5 scale)
df['R_Score'] = pd.qcut(df['Recency'].rank(method='first'), q=5, labels=[5, 4, 3, 2, 1])  # Lower recency = higher score
df['F_Score'] = pd.qcut(df['Frequency'].rank(method='first'), q=5, labels=[1, 2, 3, 4, 5])
df['M_Score'] = pd.qcut(df['Monetary'].rank(method='first'), q=5, labels=[1, 2, 3, 4, 5])

# Calculate RFM score
df['RFM_Score'] = df['R_Score'].astype(int) + df['F_Score'].astype(int) + df['M_Score'].astype(int)

print("RFM Score Distribution:")
print(df['RFM_Score'].describe())

In [ ]:
# Create RFM segments
def rfm_segment(row):
    r, f, m = row['R_Score'], row['F_Score'], row['M_Score']
    
    # Champions: High on all dimensions
    if r >= 4 and f >= 4 and m >= 4:
        return 'Champions'
    # Loyal: High frequency and monetary, any recency
    elif f >= 4 and m >= 4:
        return 'Loyal Customers'
    # Potential Loyalists: Recent with moderate frequency
    elif r >= 4 and f >= 2:
        return 'Potential Loyalists'
    # At Risk: Were good customers, not recent
    elif r <= 2 and f >= 3 and m >= 3:
        return 'At Risk'
    # Can't Lose: High value but churning
    elif r <= 2 and f >= 4 and m >= 4:
        return "Can't Lose Them"
    # Hibernating: Low recent activity, some history
    elif r <= 2 and f <= 2:
        return 'Hibernating'
    # New Customers: Recent but low frequency
    elif r >= 4 and f <= 2:
        return 'New Customers'
    else:
        return 'Need Attention'

df['RFM_Segment'] = df.apply(rfm_segment, axis=1)

# RFM segment distribution
rfm_counts = df['RFM_Segment'].value_counts()
print("\nRFM Segment Distribution:")
for seg, count in rfm_counts.items():
    print(f"  {seg}: {count} ({count/len(df)*100:.1f}%)")

In [ ]:
# Visualize RFM segments
plt.figure(figsize=(12, 6))
rfm_counts.plot(kind='bar', color='steelblue', edgecolor='black')
plt.title('Customer Distribution by RFM Segment')
plt.xlabel('Segment')
plt.ylabel('Number of Customers')
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.savefig('../reports/figures/rfm_segments.png', dpi=150, bbox_inches='tight')
plt.show()

## 5. K-Means Clustering

In [ ]:
# Prepare features for clustering
cluster_features = ['Recency', 'Frequency', 'Monetary', 'Tenure', 'NumUniqueProducts']
X_cluster = df[cluster_features].copy()

# Scale features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_cluster)

# Find optimal K using elbow method
inertias = []
K_range = range(2, 11)

for k in K_range:
    kmeans = KMeans(n_clusters=k, random_state=42, n_init=10)
    kmeans.fit(X_scaled)
    inertias.append(kmeans.inertia_)

plt.figure(figsize=(10, 5))
plt.plot(K_range, inertias, 'bo-', markersize=8)
plt.xlabel('Number of Clusters (K)')
plt.ylabel('Inertia')
plt.title('Elbow Method for Optimal K')
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.savefig('../reports/figures/elbow_method.png', dpi=150, bbox_inches='tight')
plt.show()

In [ ]:
# Apply K-Means with optimal K (4 clusters)
optimal_k = 4
kmeans = KMeans(n_clusters=optimal_k, random_state=42, n_init=10)
df['Cluster'] = kmeans.fit_predict(X_scaled)

# Cluster profile
cluster_profile = df.groupby('Cluster')[cluster_features + ['CLV']].mean()
print("K-Means Cluster Profiles:")
cluster_profile

In [ ]:
# Name clusters based on characteristics
cluster_names = {
    cluster_profile['CLV'].idxmax(): 'High Value',
    cluster_profile['CLV'].idxmin(): 'Low Value',
}
remaining = [i for i in range(optimal_k) if i not in cluster_names]
if len(remaining) >= 2:
    mid_vals = cluster_profile.loc[remaining, 'CLV'].sort_values()
    cluster_names[mid_vals.index[0]] = 'Medium-Low'
    cluster_names[mid_vals.index[1]] = 'Medium-High'

df['Cluster_Name'] = df['Cluster'].map(cluster_names)

print("\nCluster Distribution:")
print(df['Cluster_Name'].value_counts())

## 6. Segment Recommendations

In [ ]:
# Business recommendations for each segment
recommendations = {
    'High Value': {
        'Description': 'Your best customers with highest CLV',
        'Strategy': 'Retention & Loyalty',
        'Actions': [
            'VIP treatment and exclusive offers',
            'Early access to new products',
            'Personalized recommendations',
            'Loyalty program with premium rewards',
            'Dedicated customer support'
        ]
    },
    'Medium-High': {
        'Description': 'Good customers with potential to become high value',
        'Strategy': 'Growth & Upselling',
        'Actions': [
            'Cross-sell complementary products',
            'Volume discounts to increase order size',
            'Referral incentives',
            'Subscription offers for regular purchases'
        ]
    },
    'Medium-Low': {
        'Description': 'Moderate value customers needing engagement',
        'Strategy': 'Engagement & Education',
        'Actions': [
            'Email campaigns showcasing product range',
            'Bundle deals to increase basket size',
            'Seasonal promotions',
            'Product usage tips and inspiration'
        ]
    },
    'Low Value': {
        'Description': 'Low spending customers, may be new or churning',
        'Strategy': 'Activation & Re-engagement',
        'Actions': [
            'Welcome discounts for new customers',
            'Win-back campaigns for dormant customers',
            'Entry-level product recommendations',
            'Free shipping thresholds to increase orders'
        ]
    }
}

print("Segment Recommendations:")
print("=" * 60)
for segment, rec in recommendations.items():
    count = (df['CLV_Segment'] == segment).sum()
    print(f"\n{segment} ({count} customers)")
    print(f"  Description: {rec['Description']}")
    print(f"  Strategy: {rec['Strategy']}")
    print("  Actions:")
    for action in rec['Actions']:
        print(f"    - {action}")

## 7. Segment Summary Dashboard

In [ ]:
# Create summary dashboard
fig, axes = plt.subplots(2, 2, figsize=(14, 12))

# 1. Customer Distribution
segment_counts.plot(kind='pie', ax=axes[0, 0], colors=colors, autopct='%1.1f%%')
axes[0, 0].set_title('Customer Distribution')
axes[0, 0].set_ylabel('')

# 2. CLV Distribution by Segment
df.boxplot(column='CLV', by='CLV_Segment', ax=axes[0, 1])
axes[0, 1].set_title('CLV Distribution by Segment')
axes[0, 1].set_xlabel('Segment')
axes[0, 1].set_ylabel('CLV (GBP)')
plt.suptitle('')

# 3. Average Metrics by Segment
segment_avgs = df.groupby('CLV_Segment')[['Frequency', 'Monetary', 'NumUniqueProducts']].mean()
segment_avgs.plot(kind='bar', ax=axes[1, 0], color=['coral', 'steelblue', 'green'], edgecolor='black')
axes[1, 0].set_title('Average Metrics by Segment')
axes[1, 0].set_xlabel('Segment')
axes[1, 0].tick_params(axis='x', rotation=45)
axes[1, 0].legend(loc='upper left')

# 4. Revenue Contribution
revenue_pct = (segment_clv / segment_clv.sum() * 100)
revenue_pct.plot(kind='bar', ax=axes[1, 1], color='gold', edgecolor='black')
axes[1, 1].set_title('Revenue Contribution by Segment (%)')
axes[1, 1].set_xlabel('Segment')
axes[1, 1].set_ylabel('% of Total Revenue')
axes[1, 1].tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.savefig('../reports/figures/segment_dashboard.png', dpi=150, bbox_inches='tight')
plt.show()

## 8. Save Segmented Data

In [ ]:
# Save segmented customer data
output_cols = ['Customer ID', 'Recency', 'Frequency', 'Monetary', 'Tenure',
               'NumUniqueProducts', 'AvgOrderValue', 'CLV', 
               'CLV_Segment', 'RFM_Segment', 'Cluster_Name']

# Only include columns that exist
output_cols = [col for col in output_cols if col in df.columns]

output_df = df[output_cols].copy()
output_path = Path('../data/processed/customer_segments.csv')
output_df.to_csv(output_path, index=False)

print(f"Segmented data saved to: {output_path}")
print(f"Customers: {len(output_df)}")

## 9. Segmentation Summary

### Segmentation Methods Used:
1. **Quantile-Based CLV Segmentation**: 4 tiers based on predicted CLV
2. **RFM Segmentation**: Traditional marketing segments based on RFM scores
3. **K-Means Clustering**: Data-driven clusters using behavioral features

### Key Findings:
- **Pareto Principle**: Top 25% customers contribute ~X% of revenue
- **High Value segment** has highest frequency, monetary value, and product diversity
- **Low Value segment** shows opportunity for activation or win-back campaigns

### Business Recommendations:
- **High Value**: Retention focus, VIP treatment, exclusive offers
- **Medium-High**: Growth focus, upselling, cross-selling
- **Medium-Low**: Engagement focus, education, promotions
- **Low Value**: Activation focus, win-back campaigns, entry offers

### Implementation:
- Use segments for targeted email marketing
- Customize product recommendations by segment
- Allocate customer service resources based on CLV
- Design segment-specific promotions and loyalty rewards